# Section_2 project

In [29]:
import pandas as pd
import pandas_profiling
import numpy as np
import shap
from scipy.stats import randint, uniform
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('./video_games.csv')
test = pd.read_csv('./test.csv')

In [3]:
profile = data.profile_report()
profile

## 데이터 내용과 프로젝트의 목적

### 데이터
- 데이터는 게임내 표현효과(마약,술,폭력성,혈흔,욕설 등)의 유무와 게임의 esrb_rating(심의등급) 이 포함되어있다.
    - E : 전체이용가
    - ET : 10세 이상
    - T : 13세 이상
    - M : 17세(성인) 이상
- 데이터의 분석목적은 게임내 표현들을 토대로 게임의 심의등급을 예측하는것이다.
    - 'https://www.kaggle.com/imohtn/video-games-rating-by-esrb'

### 프로젝트 목적과 분석방향
- 하지만 거의 모든 선정적인 효과들의 사용여부가 13세기준으로 명확하게 구분되어있어 머신러닝을 활용하지않아도 알 수 있다
- 그러므로 선정적인 표현이 어느정도 섞여있는 17세기준으로 분석으로 바꾸어 진행하였다
    - 즉 성인/비성인 게임인지 구분하는 이진분류로 바꾸었다
- 이렇게 분석방향을 바꿈으로써 게임의 등급의 유추와 더불어  
    성인등급 판정에 큰 영향을 미치는 특성도 알 수 있다
    - 이러한 특성을 조절함으로써 의도적으로 성인/비성인 등급을 받도록 유도할 수 있다
    - 게임의 완성도가 높다는 가정하에, 성인용게임은 어느정도의 수익률이 보장되고/ 전체이용가 게임은 사용자의 폭이 넓어진다.

- 설정한 가설은 **"특정 표현효과를 조절함으로써 의도적으로 성인 or 비성인 등급을 받을 수 있을것이다"** 이다.
    - features = 게임내 표현효과들
    - target = esrb 등급

## Data Leakage 확인 및 모델의 한계

## Baseline model 선택
- 기준모델은 두가지 타겟값중 많이 판정된 등급으로하는(최빈값) 모델이다
    - 추가로 로지스틱회귀 모델도 사용
        - 정확도를 떠나, 분류 문제를 해결할때 regression과 ensemble 모델의 분석 경향을 비교해보기 위해

In [4]:
## target 값 번경하는 함수 선언
def change_target(rate):
    # 성인등급 = 1
    if rate == 'M':
        rate = 1
        return rate
    # 비성인 등급 = 0
    else:
        rate = 0
        return rate
    
# train data에 apply로 적용
data.esrb_rating = data.esrb_rating.apply(change_target)
data.drop('title',axis=1,inplace=True)
# test data에 apply로 적용
test.esrb_rating = test.esrb_rating.apply(change_target)
test.drop('title',axis=1,inplace=True)


data.shape, test.shape

((1895, 33), (500, 33))

In [5]:
# target 분포확인(최빈값이 80%이므로 accuracy 로는 분석불가능, accuracy,f1,roc_auc 모두 확인)
data.esrb_rating.value_counts()

0    1508
1     387
Name: esrb_rating, dtype: int64

In [42]:
## 최빈값 모델
mode_baseline = [0] * len(data.esrb_rating)
mode_baseline_accuracy = accuracy_score(data.esrb_rating, mode_baseline)
print('최빈값 기준모델 정확도 : ', mode_baseline_accuracy)

최빈값 기준모델 정확도 :  0.795778364116095


In [7]:
## 타겟,피쳐 설정
target = 'esrb_rating'
features = data.drop('esrb_rating', axis=1).columns

X_train = data[features]
y_train = data[target]
X_test = test[features]
y_test = test[target]

In [8]:
## 로지스틱회귀 모델
## train,val 분리
X_train_logi ,X_val_logi ,y_train_logi ,y_val_logi = train_test_split(X_train,y_train)
X_train_logi.shape, X_val_logi.shape

((1421, 32), (474, 32))

In [43]:
## 학습 및 score확인
logistic_model = LogisticRegression()

logistic_model.fit(X_train_logi,y_train_logi)

logi_val_score = logistic_model.score(X_val_logi,y_val_logi)
print('로지스틱회귀모델 검증 정확도 : ', logi_val_score)

로지스틱회귀모델 검증 정확도 :  0.9388185654008439


## Model selection

- 데이터를 train,val로 분리
- 분류문제이므로 Decision Tree, Ensemble(RandomForest,XGboost) 모델 사용
- 세 모델과 baseline 모델의 평가지표 비교
- CV를 통해 일반화될 가능성을 확인
- 세 모델의 CV 진행(RandomizedCV)
- 모델 선택

### 정규화,인코딩 사용 X
- 모든 값이 0,1로 되어있으므로 정규화와 인코딩작업은 따로 필요하지 않음
- feature 30개, 갯수 줄여줘야함

In [10]:
# 데이터 분리
X_train,X_val ,y_train ,y_val = train_test_split(X_train,y_train)
X_train.shape, X_val.shape

((1421, 32), (474, 32))

### Decision Tree

In [46]:
## DecisionTree
pipe_dt = make_pipeline(
    DecisionTreeClassifier(random_state=10)
)

In [47]:
## CV
dists = {
    'decisiontreeclassifier__max_depth' : [3,4,5,6,7], 
    'decisiontreeclassifier__max_features': [3,4,5,6] 
}

clf_dt = RandomizedSearchCV(
    pipe_dt, 
    param_distributions=dists, 
    n_iter=200, 
    cv=5,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

clf_dt.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.8s finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('decisiontreeclassifier',
                                              DecisionTreeClassifier(random_state=10))]),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'decisiontreeclassifier__max_depth': [3,
                                                                              4,
                                                                              5,
                                                                              6,
                                                                              7],
                                        'decisiontreeclassifier__max_features': [3,
                                                                                 4,
                                                                                 5,
                                                                                 6]},
                   scoring=

In [48]:
val_pred = clf_dt.predict(X_val)

## 검증세트 정확도 확인
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97       386
           1       0.82      0.92      0.87        88

    accuracy                           0.95       474
   macro avg       0.90      0.94      0.92       474
weighted avg       0.95      0.95      0.95       474



In [15]:
## 검증세트 AUC 확인
y_pred_proba = clf_dt.predict_proba(X_val)[:, 1]
dt_auc_val = roc_auc_score(y_val, y_pred_proba)
dt_auc_val

0.9778173575129533

### RandomForest

In [17]:
## RandomForest
pipe_rf = make_pipeline(
    RandomForestClassifier(random_state=10,n_jobs=-1)
)

In [18]:
## CV
dists = {
    'randomforestclassifier__n_estimators': randint(50,200), 
    'randomforestclassifier__max_depth' : [4,5,6,7,8,9], 
    'randomforestclassifier__max_features': [4,5,6,7] 
}

clf_rf = RandomizedSearchCV(
    pipe_rf, 
    param_distributions=dists, 
    n_iter=200, 
    cv=5,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

clf_rf.fit(X_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   46.1s finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('randomforestclassifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=10))]),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': [4,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9],
                                        'randomforestclassifier__max_features': [4,
                                                                                 5,
                                    

In [19]:
## 최적파라미터 확인
clf_rf.best_estimator_

Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(max_depth=9, max_features=5,
                                        n_estimators=53, n_jobs=-1,
                                        random_state=10))])

In [49]:
val_pred = clf_rf.predict(X_val)

## 검증세트 정확도 확인
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       386
           1       0.88      0.93      0.91        88

    accuracy                           0.96       474
   macro avg       0.93      0.95      0.94       474
weighted avg       0.97      0.96      0.96       474



In [21]:
## 검증세트 AUC 확인
y_pred_proba = clf_rf.predict_proba(X_val)[:, 1]
rf_auc_val = roc_auc_score(y_val, y_pred_proba)
rf_auc_val

0.9864725624116816

### XGBClassifier

In [23]:
y_train.value_counts(normalize=True)

0    0.789585
1    0.210415
Name: esrb_rating, dtype: float64

In [24]:
## XGBClassifier

ratio =  0.209711/0.790289
xgb = XGBClassifier(n_estimators=500,
                    max_depth=6,
                    learning_rate=0.2,
                    scale_pos_weight=ratio,
                    n_jobs=-1)

In [25]:
## XGB 최적화
eval_set = [(X_train,y_train),
            (X_val,y_val)]

xgb.fit(X_train, y_train,
       eval_set= eval_set,
       eval_metric='auc',
       early_stopping_rounds=20
       )

[0]	validation_0-auc:0.94503	validation_1-auc:0.95176
[1]	validation_0-auc:0.94810	validation_1-auc:0.95220
[2]	validation_0-auc:0.94853	validation_1-auc:0.95449
[3]	validation_0-auc:0.94934	validation_1-auc:0.95571
[4]	validation_0-auc:0.98556	validation_1-auc:0.98191
[5]	validation_0-auc:0.98565	validation_1-auc:0.98197
[6]	validation_0-auc:0.98596	validation_1-auc:0.98173
[7]	validation_0-auc:0.98645	validation_1-auc:0.98185
[8]	validation_0-auc:0.98620	validation_1-auc:0.98212
[9]	validation_0-auc:0.98689	validation_1-auc:0.98242
[10]	validation_0-auc:0.98690	validation_1-auc:0.98287
[11]	validation_0-auc:0.98689	validation_1-auc:0.98304
[12]	validation_0-auc:0.98655	validation_1-auc:0.98323
[13]	validation_0-auc:0.98583	validation_1-auc:0.98209
[14]	validation_0-auc:0.98658	validation_1-auc:0.98341
[15]	validation_0-auc:0.98670	validation_1-auc:0.98318
[16]	validation_0-auc:0.98679	validation_1-auc:0.98318
[17]	validation_0-auc:0.98680	validation_1-auc:0.98315
[18]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.26535988733235566,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [26]:
print('XGBClassifier 검증 정확도: ', xgb.score(X_val,y_val))

XGBClassifier 검증 정확도:  0.959915611814346


In [27]:
## 검증세트 AUC 확인
y_pred_proba = xgb.predict_proba(X_val)[:, 1]
xgb_auc_val = roc_auc_score(y_val, y_pred_proba)
xgb_auc_val

0.9874146255299106

In [30]:
f1_score(y_val, xgb.predict(X_val))

0.8862275449101796

## 모델 검증정확도 비교 및 모델 최종선택
### f1_score / accuracy / roc_auc
- DecisionTree
    - 0.78 / 0.93 / 0.98 
- RandomForest
    - 0.90 / 0.96 / 0.99 
- XGBClassifier
    - 0.81 / 0.94 / 0.99 
- RandomForest가 가장 정확

## 모델의 분석 시각화 (SHAP)
- 분류모델의 시각화 SHAP
    - 이 과정을통해 target에 대한 feature들의 영향력을 확인
- 어떤 feature를 사용해야 성인/비성인 등급을 받도록 유도할수 있는지 파악
    - 가설검증 가능
- 추가로 더 필요한 특성은 무엇이 있고 어떻게 구할 수 있을까?
    - 피/마약/알콜이 모두 사용됨의 여부를 포함한 특성이 있다면 분석과정을 줄일 수 있음(해당 특성들을 합쳐 새 feature로 만든 후 정규화해주어 분석가능)
    - 만약 게임의 판매량 데이터도 있다면, 심의등급에 따른 판매량을 비교하여 수익을 극대화하는 게임의 방향 설정가능


### 특성중요도 파악하기

In [33]:
import eli5
from eli5.sklearn import PermutationImportance

#pipe
pipe = make_pipeline(
    RandomForestClassifier(max_depth=9,
                           max_features=4,
                           n_estimators=138,
                           n_jobs=-1,
                           random_state=10)
)
pipe.fit(X_train,y_train)

# permuter 정의
permuter = PermutationImportance(
    pipe.named_steps['randomforestclassifier'],
    scoring='f1',
    n_iter=5,
    random_state=10
)

permuter.fit(X_val, y_val);

feature_names = X_val.columns.tolist()

# 특성별 score 확인
eli5.show_weights(
    permuter, 
    top=15,
    feature_names=feature_names
)

Weight,Feature
0.3893 ± 0.0417,strong_janguage
0.1221 ± 0.0172,blood_and_gore
0.0653 ± 0.0299,strong_sexual_content
0.0183 ± 0.0047,alcohol_reference
0.0147 ± 0.0196,no_descriptors
0.0147 ± 0.0386,blood
0.0131 ± 0.0049,lyrics
0.0113 ± 0.0078,violence
0.0108 ± 0.0189,mild_lyrics
0.0099 ± 0.0049,mild_suggestive_themes


### 등급선정에 영향을 주는 feature 찾아내기
- 성인등급을 받은 게임하나를 예시로 시각화

In [60]:
## 예시로 확인해보기(성인등급을 받은 test샘플)
row = X_test.iloc[[6]]

In [61]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(row)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row, 
    link='logit' # SHAP value를 확률로 변환해 표시합니다.
)

In [63]:
# 100개 테스트 샘플에 대해서 각 특성들의 영향확인
shap_values = explainer.shap_values(X_test.iloc[:400])
shap.force_plot(explainer.expected_value, shap_values, X_test.iloc[:400])

## Recheck point
    
- 모델의 한계점과 유용성을 정확하게설명할수 있게 정리하기
    - 랜덤포레스트의 강점과 약점
    

- 시각화,permutation importance 기반으로 중요한 feature 찾아서 낮은feature 제거하고 다시 모델학습,검증,테스트 해보기
- 결과를 바탕으로 가장 영향력이 큰 feature를 확인해서 가설검증하기

In [ ]:
## 영향력 상위10개의 feature 만 사용해서 다시 모델학습 해보기